#### Restart & Run All Then Call Ord-Log

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 9, 24)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,30,S,WHART,"10,000",11.10,1,C10.5,SET,2022-08-18


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade     n...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,BANPU,7500,13.00,1,CP1S,SET50,2022-09-14
1,5,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
2,7,B,JMART,3000,48.00,1,1L,SET50,2022-04-19
3,8,B,JMT,1200,72.00,1,1L,SET50,2022-04-19
4,13,B,PTT,4500,35.00,1,1L,SET50,2022-09-28
5,18,B,TOP,3000,54.00,1,1L,SET50,2022-02-02
6,23,S,IVL,2400,44.25,0,DOS,SET50,2022-08-30
7,26,S,PTTGC,3000,51.50,0,C51.25,SET50,2022-09-02


In [6]:
name = 'JMT'
limit = 20 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,20.00,20.00,20.00,20.00,20.00
mean,75.86,76.69,75.12,7312785.35,75.92
std,1.90,1.58,1.81,2539111.07,1.70
min,72.25,73.75,71.75,2924941.00,72.50
25%,74.25,75.19,73.44,5067110.50,74.88
50%,76.50,77.12,75.38,7279666.50,76.25
75%,77.31,78.00,76.50,9471371.25,77.06
max,78.50,78.75,77.50,11339288.00,78.50


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,BAM,6000,16.0,0,XXX,SET100,2022-04-29
1,4,B,BCP,3000,29.0,0,XXX,SET100,2022-02-02
2,12,B,PSL,6000,14.0,0,3L,SET100,2022-02-02
3,14,B,SINGER,3000,35.5,0,52WL,SET100,2022-02-02
4,15,B,SPALI,6000,18.5,1,3L,SET100,2022-08-23
5,17,B,STARK,25000,3.9,1,3L,SET100,2022-02-02
6,20,S,BCH,3000,20.6,0,C20.50,SET100,2022-08-31
7,21,S,DOHOME,2800,20.3,0,RD20pct,SET100,2022-02-02
8,28,S,STA,2500,26.0,0,CP1S,SET100,2022-08-23


In [8]:
name = 'BAM'
limit = 60 # 3 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,17.49,17.67,17.29,14319727.47,17.50
std,0.75,0.71,0.74,7771905.93,0.72
min,16.10,16.30,15.90,2394120.00,16.20
25%,16.90,17.20,16.68,9513159.25,16.95
50%,17.50,17.80,17.30,12786353.00,17.70
75%,18.20,18.40,18.10,17009113.00,18.20
max,18.60,18.60,18.30,43530805.00,18.50


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,33.50,1,52WL,SET,2022-03-03
1,6,B,CPNREIT,10000,18.80,1,HD,SET,2022-08-17
2,9,B,KSL,30000,3.30,0,6L,SET,2022-02-02
3,10,B,LANNA,3000,20.80,0,3L,SET,2022-02-02
4,11,B,OISHI,2400,42.00,0,6L,SET,2022-02-02
5,16,B,SSP,6000,8.50,0,52WL,SET,2022-02-02
6,19,S,ASP,10000,3.82,0,CP1S,SET,2022-08-24
7,22,S,GVREIT,10000,9.20,1,DOS,SET,2022-08-26
8,24,S,JASIF,10000,9.20,0,CP1S,SET,2022-08-19
9,25,S,MCS,25000,12.90,0,C12.8,SET,2022-09-03


In [10]:
name = 'LANNA'
limit = 120 # 6 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,20.63,20.92,20.31,4884972.07,20.61
std,1.71,1.72,1.65,4653737.41,1.68
min,17.80,18.40,17.80,652814.00,18.30
25%,19.38,19.50,19.10,2248199.25,19.38
50%,20.00,20.35,19.65,3553441.00,20.00
75%,21.95,22.42,21.52,5297324.75,21.88
max,24.40,24.60,23.90,27453276.00,24.30


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,33.50,1,52WL,SET,2022-03-03
1,2,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
2,3,B,BANPU,7500,13.00,1,CP1S,SET50,2022-09-14
3,4,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
4,5,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
5,6,B,CPNREIT,10000,18.80,1,HD,SET,2022-08-17
6,7,B,JMART,3000,48.00,1,1L,SET50,2022-04-19
7,8,B,JMT,1200,72.00,1,1L,SET50,2022-04-19
8,9,B,KSL,30000,3.30,0,6L,SET,2022-02-02
9,10,B,LANNA,3000,20.80,0,3L,SET,2022-02-02
